In [669]:
#Imports
import numpy as np
from scipy.io import wavfile
from scipy.io import wavfile
import matplotlib.pyplot as plt

In [670]:
#Renvoie la liste des notes d'un piano avec leur fréquences respective
def get_piano_notes():   
    # White keys are in Uppercase and black keys (sharps) are in lowercase
    octave = ['C', 'c', 'D', 'd', 'E', 'F', 'f', 'G', 'g', 'A', 'a', 'B'] 
    base_freq = 440 #Frequency of Note A4
    keys = np.array([x+str(y) for y in range(0,9) for x in octave])
    # Trim to standard 88 keys
    start = np.where(keys == 'A0')[0][0]
    end = np.where(keys == 'C8')[0][0]
    keys = keys[start:end+1]
    
    note_freqs = dict(zip(keys, [2**((n+1-49)/12)*base_freq for n in range(len(keys))]))
    note_freqs[''] = 0.0 # stop
    return note_freqs

In [671]:
#Récupère le son à partir de la fréquance et du temps donné
def get_sine_wave(frequency, duration, sample_rate=44100, amplitude=4096):
    t = np.linspace(0, duration, int(sample_rate*duration)) # Time axis
    wave = amplitude*np.sin(2*np.pi*frequency*t)
    return wave

In [672]:
#Liste des notes
note_freqs = get_piano_notes()

In [673]:
#Créer l'accord à partir des notes données
def create_note(note, BPM):
    res = 0
    note = note.split("-")
    duration = len(note)/4
    for i in range(0,len(note[0]),2):
        res += get_sine_wave(note_freqs[note[0][i:i+2]], duration=duration*(60/BPM))
    return res

In [674]:
#analyse chaque accord d'une ligne
def analyse_notes(notes, BPM):
    for i in range(len(notes)):
        notes[i] = create_note(notes[i], BPM)
    return notes

In [675]:
#Lis une partition
def read_partition(partition_path, BPM):
    partition = []
    with open(partition_path) as f:
        contents = f.readlines()
        for lines in contents:
            res = lines.split()
            for elt in analyse_notes(res, BPM):
                partition.append(elt)
    return partition

In [677]:
#Exemple avec GigaChad
partition = read_partition("GigaChad.txt", 100)
music = np.concatenate(partition)
wavfile.write('GigaChad.wav', rate=44100, data=music.astype(np.int16))